In [1]:
# External libraries
import numpy as np
from matplotlib import pyplot as plt

# Drake dependencies
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    MultibodyPlant,
    Demultiplexer,
    DiscreteContactApproximation,
    ConstantVectorSource,
    Parser,
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    JointSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    MultibodyPositionToGeometryPose,
    Multiplexer,
    Parser,
    PrismaticJoint,
    SceneGraph,
    SpatialInertia,
    Sphere,
    UnitInertia,
    MeshcatVisualizerParams,
    LoadModelDirectivesFromString,
    ProcessModelDirectives,
    AddDefaultVisualization
)

# Custom classes and functions
from ShishKebot.CartesianStiffnessController import CartesianStiffnessController

# Helper functions
import manipulation
from manipulation.meshcat_utils import MeshcatSliders, StopButton
from manipulation.scenarios import AddShape
from manipulation.utils import RenderDiagram
from manipulation.scenarios import AddRgbdSensors
import os
import sys 

/home/trevor/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
meshcat = StartMeshcat()
meshcat.SetProperty("/Background", "visible", False)
meshcat.SetProperty("/Cameras/default/rotated/<object>", "zoom", 10.5)

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
iiwa1_directive = f"""
directives:
- add_model:
    name: iiwa1
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.0]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [0.0]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [0.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        translation: [0, 0.5, 0]
- add_model:
    name: wsg1
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}

- add_model:
    name: skewer
    file: file://{os.getcwd()}/Models/skewer_5mm.sdf
    # default_free_body_pose:
    #     skewer_5mm:
    #         translation: [0.05, 0.2, 2]         
- add_weld:
    parent: wsg1::body
    child: skewer::skewer_5mm
    X_PC:
        translation: [0, 0, 0]
        rotation: !Rpy {{ deg: [270, 0, 0]}}
"""

iiwa2_directive = f"""
directives:
- add_model:
    name: iiwa2
    file: package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.6]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.75]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.0]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: world
    child: iiwa2::iiwa_link_0
    X_PC:
        translation: [0, -0.5, 0]
- add_model:
    name: wsg2
    file: package://manipulation/schunk_wsg_50_welded_fingers.sdf
- add_weld:
    parent: iiwa2::iiwa_link_7
    child: wsg2::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}
"""

world_directive = f"""
directives:
- add_model:
    name: table
    file: file://{os.getcwd()}/Models/ground.sdf
- add_weld:
    parent: world
    child: table::base

- add_model:
    name: cube
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, 0]}
- add_model:
    name: cube2
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, .1]}
- add_model:
    name: cube3
    file: file://{os.getcwd()}/Models/cube_food.sdf
    default_free_body_pose:
        cube_food:
            rotation: !Rpy {{ deg: [{np.random.rand()*180}, {np.random.rand()*180}, {np.random.rand()*180}]}}
            translation: {[-0.4-np.random.rand()*0.6, np.random.rand() - 1, .2]}

- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        # rotation: !Rpy {{ deg: [180, 0, 0]}}
        # translation: [0, 0, 4]
        rotation: !Rpy {{ deg: [225, 0, 0]}}
        translation: [-0.7, -1.5, 0.5]
- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera0_origin
    child: camera0::base

- add_frame:
    name: camera1_origin
    X_PF:
        base_frame: world
        rotation: !Rpy {{ deg: [135, 0, 0]}}
        translation: [-0.7, 0.5, 0.5]
- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf
- add_weld:
    parent: camera1_origin
    child: camera1::base
"""

In [11]:
builder = DiagramBuilder()

In [12]:
# Add iiwa1 to the scene
iiwa1_plant, iiwa1_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
iiwa1_plant.set_name("iiwa1_plant")
iiwa1_scene_graph.set_name("iiwa1_scene_graph")
iiwa1_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
directives = LoadModelDirectivesFromString(iiwa1_directive)
parser = Parser(iiwa1_plant)
parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
models = ProcessModelDirectives(directives, iiwa1_plant, parser)
iiwa1 = iiwa1_plant.GetModelInstanceByName("iiwa1")
wsg1 = iiwa1_plant.GetModelInstanceByName("wsg1")
iiwa1_plant.Finalize()

# Meshcat
params = MeshcatVisualizerParams()
params.prefix = "iiwa1"
MeshcatVisualizer.AddToBuilder(builder, iiwa1_scene_graph, meshcat, params)

In [13]:
# Add iiwa2 to the scene
iiwa2_plant, iiwa2_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
iiwa2_plant.set_name("iiwa2_plant")
iiwa2_scene_graph.set_name("iiwa2_scene_graph")
iiwa2_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
directives = LoadModelDirectivesFromString(iiwa2_directive)
parser = Parser(iiwa2_plant)
parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
models = ProcessModelDirectives(directives, iiwa2_plant, parser)
iiwa2 = iiwa2_plant.GetModelInstanceByName("iiwa2")
wsg2 = iiwa2_plant.GetModelInstanceByName("wsg2")
iiwa2_plant.Finalize()

# Meshcat
params = MeshcatVisualizerParams()
params.prefix = "iiwa2"
MeshcatVisualizer.AddToBuilder(builder, iiwa2_scene_graph, meshcat, params)

In [14]:
# Add world models to the scene
world_plant, world_scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-3)
world_plant.set_name("world_plant")
world_scene_graph.set_name("world_scene_graph")
world_plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
directives = LoadModelDirectivesFromString(world_directive)
parser = Parser(world_plant)
parser.package_map().Add("manipulation", manipulation.__path__[0] + "/models/")
models = ProcessModelDirectives(directives, world_plant, parser)
world_plant.Finalize()

# Meshcat
params = MeshcatVisualizerParams()
params.prefix = "world"
MeshcatVisualizer.AddToBuilder(builder, world_scene_graph, meshcat, params)

# Add the cameras to the diagram
AddRgbdSensors(builder, world_plant, world_scene_graph)

In [ ]:
# Add our torque controller
controller: CartesianStiffnessController = builder.AddSystem(CartesianStiffnessController(plant, "iiwa1", "wsg1"))
controller.SetGains(
    position=(10.0, 5.0),
    orientation=(10.0, 5.0),
    null_space=5.0
)

# Wire up controller torque to iiwa torque
builder.Connect(controller.GetOutputPort("iiwa_torque_cmd"), plant.get_actuation_input_port(iiwa))

# Extract state from plant and feed to controller
state_demultiplexer = builder.AddSystem(Demultiplexer([7,7]))
builder.Connect(plant.get_state_output_port(iiwa), state_demultiplexer.get_input_port(0))
builder.Connect(state_demultiplexer.get_output_port(0), controller.GetInputPort("iiwa_position_measured"))
builder.Connect(state_demultiplexer.get_output_port(1), controller.GetInputPort("iiwa_velocity_measured"))


In [9]:
diagram = builder.Build()
simulator = Simulator(diagram)

In [10]:
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(np.inf)

KeyboardInterrupt: 